<a href="https://colab.research.google.com/github/ErnestasSku/UniversityPrograms/blob/main/3rd%20Year/Deep%20Learning/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ernestas Škudzinskas 2016049 
Programų sistemos
3 kursas 4 grupė

In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

import cv2 as cv

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Dataset"

def fp_to_nr(x):
  x = x[:-4]
  return int(x[6:])

files = os.listdir(path)
files.sort(key=fp_to_nr)

In [5]:
def my_gen():
  for file in files:
    yield cv.resize(cv.imread(f"{path}/{file}"), (224, 224))

In [6]:
net = tf.keras.applications.resnet50.ResNet50()

102967424/102967424 [==============================] - 1s 0us/step


In [7]:
input = tf.keras.Input(shape=(224, 224, 3))
x = net(input)
model = tf.keras.Model(inputs=input, outputs=x)

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [9]:
output_types = tf.float32
output_shapes = (224,224,3)
data = tf.data.Dataset.from_generator(my_gen, output_types=output_types, output_shapes=output_shapes)
data = data.cache()
data = data.batch(32)
data = data.prefetch(tf.data.AUTOTUNE)

In [10]:
p = model.predict(data)

32/32 [==============================] - 187s 5s/step


In [11]:
candles = [470]
coffee = [967, 504, 505]
panda = [387, 388]

In [21]:
candleCount = 0
coffeeCount = 0
pandaCount = 0

threshold = 0.8

filterFunc = lambda i : list(filter(lambda x : x[i] > threshold, p))
sumFilter = lambda arr : sum([len(filterFunc(i)) for i in arr])

totalCandleCount = sumFilter(candles)
totalCoffeeCount = sumFilter(coffee)
totalPandaCount = sumFilter(panda)


print(f"total candles found: {totalCandleCount}")
print(f"total pandas found: {totalPandaCount}")
print(f"total coffees found: {totalCoffeeCount}")

def prediction_to_symbol(n: np.array) -> str:
  if n[470] > threshold:
    return 'ca'
  elif n[967] > threshold or n[504] > threshold or n[505] > threshold:
    return 'co'
  elif n[387] > threshold or n[388] > threshold:
    return 'p'   
  else: 
    return 'n'

mapped = list(map(prediction_to_symbol, p))
correct = ['ca'] * 333 + ['co'] * 333 + ['p'] * 333

print('')
print(f" Accuracy: {accuracy_score(correct, mapped)}")
print(f" Precision: {precision_score(correct, mapped, average='macro')}")
print(f" F1: {f1_score(correct, mapped, average='macro')}")
 



total candles found: 216
total pandas found: 132
total coffees found: 60

 Accuracy: 0.4064064064064064
 Precision: 0.7476851851851851
 F1: 0.4131711795842007


Some texting bellow